This is an example using [rdflib](https://rdflib.readthedocs.io/), which is a Python Library that implements reading and writing RDF and SPARQL queries.

# Permitted Uses

In [1]:
import rdflib
g_uses = rdflib.Graph()
# load the graph related to the permitted uses
g_uses.parse("permits_use2.ttl")

<Graph identifier=N6f2f0c1e242d48e3aa9b2ce7d7eba4df (<class 'rdflib.graph.Graph'>)>

In [2]:
# number of triples in the graph?
len(g_uses)

228

### 1. Query for "Which zoning districts allow a recreation center?"

In [3]:

sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "recreation centers" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): R1
Zoning District (Label): R2
Zoning District (Label): R3


### 2. Query for "Which zoning districts permit duplexes?"
These are listed as "Two-family dwellings"

In [4]:
sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "Two-family dwellings" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): R2
Zoning District (Label): R3


### 3. Which zoning district allows restaurants?

In [5]:
sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "restaurants" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): C1
Zoning District (Label): C2
Zoning District (Label): C3
Zoning District (Label): C4


### 4. Are automotive sales allowed in a C-3 zoning district?

In [42]:
# This is more litterly ASKing if there is a triple (or set of triples existing in the dataset)
# matching atleast once.

sparql_query_auto_sales_ask1 = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

ASK {
		:c3 :permitsUse "automotive sales" .
}
"""

results = g_uses.query(sparql_query_auto_sales_ask1)

results.askAnswer


True

In [43]:
sparql_query_auto_sales_ask2 = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

ASK {
		?zoning :permitsUse "automotive sales" ;
                rdfs:label "C3" .
}
"""

results = g_uses.query(sparql_query_auto_sales_ask2)

results.askAnswer

True

# 5. I would like to build an indoor theater.  Which zoning districts permit that? 

In [35]:
sparql_query_theater = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "indoor theaters" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query_theater)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): C3
Zoning District (Label): C4


# "Bulk" Dimensional Requirements

In [16]:
import rdflib
g_bulk = rdflib.Graph()
# load the graph related to the permitted uses
g_bulk.parse("bulk.ttl")

# this is the version with units
g_bulk_units = rdflib.Graph()
# load the graph related to the permitted uses
g_bulk_units.parse("bulk2.ttl")

<Graph identifier=N2a4981fe16c340a2b2c55d1e4c126574 (<class 'rdflib.graph.Graph'>)>

In [8]:
# number of triples in the graph?
len(g_bulk)

133

### 6. What is the minimum lot size in the R1a zoning district?

In [8]:
# This isn't how you would want to interact with this.
sparql_query_min_lot_terse = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_lot_size

WHERE {
		:r1a :minLotSize ?min_lot_size .
}
"""
results_min_lot1 = g_bulk.query(sparql_query_min_lot_terse)

for row in results_min_lot1:
    print(row.min_lot_size)

35000


In [10]:
sparql_query_min_lot = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_lot_size
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :minLotSize ?min_lot_size ;
                rdfs:label "R1a" .
}
"""

results = g_bulk.query(sparql_query_min_lot)

for row in results:
    print(row.min_lot_size)
    # print(f'Zoning District (Label): {row.zoning_label}')

35000


In [18]:
for row in results:
    print(row)

(rdflib.term.Literal('35000', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


### 6b. Is the minimum lot size for property in the R1a zoning district 35,000 square feet?

In [38]:
sparql_query_min_lot_ask ="""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

ASK {
		?zoning :minLotSize 35000 ;
                rdfs:label "R1a" .
}
"""

results = g_bulk.query(sparql_query_min_lot_ask)
results.askAnswer

True

### 7. What is the minimum lot width for the R1c Zoning District? 

In [11]:
# terse query, not as useful
query_min_lot_width_terse = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?lot_width

WHERE {
	:r1c :minLotWidth ?lot_width .
}
"""

results = g_bulk.query(query_min_lot_width_terse)

for row in results:
    print(row.lot_width)

75


In [13]:
# query based on zoning label "R1c"
query_min_lot_width = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?lot_width

WHERE {
	?zoning :minLotWidth ?lot_width ;
            rdfs:label   "R1c" .
}
"""

results = g_bulk.query(query_min_lot_width)

for row in results:
    print(row.lot_width)

75


### 8. What is the maximum building height in the FI2 District?

In [15]:
# query based on zoning label "FI2"
query_max_building_height = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?max_building_height

WHERE {
	?zoning :maxBuildingHeight ?max_building_height ;
            rdfs:label   "FI2" .
}
"""

results = g_bulk.query(query_max_building_height)

for row in results:
    print(row.max_building_height)

80


In [18]:
# same query as above querying the version with units
results = g_bulk_units.query(query_max_building_height)

for row in results:
    print(row.max_building_height)

80 [ft_i]


### 9. What is the minimum front setback in the R3b zoning district? 

In [28]:
# query based on zoning label "FI2"
query_min_front_setback = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_front_setback

WHERE {
	?zoning :minFrontSetback ?min_front_setback ;
            rdfs:label   "R3b" .
}
"""

results = g_bulk_units.query(query_min_front_setback)

for row in results:
    print(row.min_front_setback)

15 [ft_i]


### 10. What is the minimum lot depth in an A1 zoning district?

In [29]:
# query based on zoning label "FI2"
query_min_lot_depth = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_lot_depth

WHERE {
	?zoning :minLotDepth ?min_lot_depth ;
            rdfs:label   "A1" .
}
"""

results = g_bulk_units.query(query_min_lot_depth)

for row in results:
    print(row.min_lot_depth)

600 [ft_i]
